In [1]:
import pandas as pd
import os

In [34]:
current_dir = os.getcwd()
results = pd.read_csv(current_dir + '/results_dropout(0.6)-confidNet-scaled-dropout(0.6)-batchsize(32).csv')

In [35]:
'''MISA-ConfidNet results
columns
    id: id of the video
    input_sentence: actual text input
    label: ground truth label list
    prediction: predicted label list
    confidence: confidence of the prediction
    confidence-t: confience while removing the text modality
    confidence-a: confience while removing the audio modality
    confidence-v: confience while removing the video modality
labels
    0: happiness
    1: sadness
    2: anger
    3: fear
    4: disgust
    5: surprise
'''
results.head()

,id,input_sentence,label,prediction,confidence,confidence-t,confidence-v,confidence-a,prediction-dynamic
0,-6rXp3zJ3kc[0],future and it's a retirement future that can u...,[0. 1. 1. 0. 0. 0.],[1. 0. 0. 0. 0. 0.],0.477915,0.462046,0.567937,0.447845,[1. 0. 0. 0. 0. 0.]
1,-6rXp3zJ3kc[1],very aware of many people are not until they g...,[0. 1. 0. 0. 0. 0.],[1. 0. 0. 1. 0. 0.],0.477771,0.330803,0.666457,0.439057,[1. 0. 0. 1. 0. 0.]
2,-6rXp3zJ3kc[2],so the answer to the question can i contribute...,[0. 1. 0. 0. 0. 0.],[0. 1. 1. 0. 0. 0.],0.488666,0.542764,0.539348,0.351203,[0. 1. 1. 0. 0. 0.]
3,-9y-fZ3swSY[0],is you can say hey i really like baby skin the...,[1. 0. 0. 0. 0. 0.],[1. 0. 0. 0. 0. 0.],0.472802,0.447487,0.931348,0.422179,[1. 0. 0. 0. 0. 0.]
4,-9y-fZ3swSY[1],face so nice by giving a good example and not ...,[1. 0. 0. 0. 0. 0.],[1. 0. 1. 1. 0. 0.],0.478402,0.455495,0.543655,0.650362,[1. 0. 1. 0. 0. 0.]


In [ ]:
text_is_noise = results[results['confidence-t'] > results['confidence']]
audio_is_noise = results[results['confidence-a'] > results['confidence']]
video_is_noise = results[results['confidence-v'] > results['confidence']]

In [ ]:
# Modality noise counts
print("Total counts: ", results.shape[0])
print("Textual noise counts: ", text_is_noise.shape[0])
print("Visual noise counts: ", video_is_noise.shape[0])
print("Audio noise counts: ", audio_is_noise.shape[0])

In [36]:
results['label'].loc[lambda x: x =='[0. 0. 0. 0. 0. 0.]'].count()

701

In [38]:
# Drop the rows with no label
results = results[results['label'] != '[0. 0. 0. 0. 0. 0.]']
print(results.shape[0])

3953


In [39]:
text_is_noise = results[results['confidence-t'] > results['confidence']]
audio_is_noise = results[results['confidence-a'] > results['confidence']]
video_is_noise = results[results['confidence-v'] > results['confidence']]

In [40]:
# Modality noise counts with no label rows dropped
print("Total counts: ", results.shape[0])
print("Textual noise counts: ", text_is_noise.shape[0])
print("Visual noise counts: ", video_is_noise.shape[0])
print("Audio noise counts: ", audio_is_noise.shape[0])

Total counts:  3953
Textual noise counts:  2525
Visual noise counts:  2954
Audio noise counts:  2081
